## 2023 Open Science Data Challenge - Level 1
## RVI Data Gathering From Sentinel 1
## Team: Devashish Mahajan

### Team Members:
### 1) Devashish Mahajan
### 2) Rohit Chaudhari

This notebook collects Radar Vegetation Index (RVI) data from Sentinel-1 radar data. Function "get_sentinel_data" return RVI for given latitude and longitude.


In [1]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')

# Import common GIS tools
import pandas as pd
from tqdm import tqdm
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio.features

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer as pc
import xrspatial.multispectral as ms
import odc
from odc.stac import stac_load

# Pass your API key here
#pc.settings.set_subscription_key('***')

### Radar Vegetation Index (RVI)

The <b>Radar Vegetation Index (RVI)</b> is one example of an index used to measure vegetation growth with radar data. Since radar data can penetrate clouds, such indices are valuable to monitor crop phenology (growth). For more information on the RVI formula, see the following Sentinel Hub link: <a href="https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-1/radar_vegetation_index/" target="_blank"><b>HERE</b></a>.


In [2]:
# Read crop location trainig data file
crop_presence_data = pd.read_csv("Crop_Location_Data_20221201.csv") 
print(crop_presence_data.info())
print(crop_presence_data['Class of Land'].value_counts())
print(crop_presence_data.head())
print(crop_presence_data.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Latitude and Longitude  600 non-null    object
 1   Class of Land           600 non-null    object
dtypes: object(2)
memory usage: 9.5+ KB
None
Rice        300
Non Rice    300
Name: Class of Land, dtype: int64
                     Latitude and Longitude Class of Land
0   (10.323727047081501, 105.2516346045924)          Rice
1  (10.322364360592521, 105.27843410554115)          Rice
2  (10.321455902933202, 105.25254306225168)          Rice
3  (10.324181275911162, 105.25118037576274)          Rice
4  (10.324635504740822, 105.27389181724476)          Rice
                       Latitude and Longitude Class of Land
595  (10.013942985253381, 105.67361318732796)      Non Rice
596   (10.01348875642372, 105.67361318732796)      Non Rice
597  (10.013034527594062, 105.67361318732796)    

In [3]:
def get_sentinel_data(latlong,time_slice,assets):
    '''
    Returns RVI values for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the RVI values have to be extracted
    assets - A list of bands to be extracted
    '''

    lat_long=latlong.replace('(','').replace(')','').replace(' ','').split(',')

    ## Hyperparameter
    box_size_deg = 0.0004 


    min_lon = float(lat_long[1])-box_size_deg/2
    min_lat = float(lat_long[0])-box_size_deg/2
    max_lon = float(lat_long[1])+box_size_deg/2
    max_lat = float(lat_long[0])+box_size_deg/2

    bbox = (min_lon, min_lat, max_lon, max_lat)
    
    time_of_interest = time_slice  
    
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time_of_interest)
    items = list(search.get_all_items()) # This produces a list of scene IDs
    
    
    # Define the pixel resolution for the final product
    # Define the scale according to our selected crs, so we will use degrees

    resolution = 10  # meters per pixel 
    scale = resolution / 111320.0 # degrees per pixel for crs=4326 
    
    # Load the data using Open Data Cube
    data = stac_load(items,bands=["vv", "vh"], patch_url=pc.sign,
                     bbox=bbox, crs="EPSG:4326", resolution=scale)
    
    # Calculate the mean of the data across the sample region
    mean = data.mean(dim=['latitude','longitude']).compute()
    
    # Calculate RVI
    dop = (mean.vv / (mean.vv + mean.vh))
    m = 1 - dop
    rvi = (np.sqrt(dop))*((4*mean.vh)/(mean.vv + mean.vh))
    #print("RVI")
    #print(rvi.to_numpy())
    
    return rvi.to_numpy()

## Note: 

All of the training and test data will assume triple cropping (3 cycles per year)
with a focus on the Winter-Spring 2021-2022 season (November to April)
and the Summer-Autumn 2022 season (April to August) 
### November 2021 to April 2022    ==  2021-11-01 to 2022-04-30
### April 2022 to August 2022  ==  2022-04-01 to 2022-08-31
### Combining above both seasons  2021-11-01 to 2022-08-31

Planting occurs within the first 2 months of each cycle (depending on location) 
and harvesting occurs within the last 2 months of each cycle"""

## RVI Data Gathering for "crop_presence_data" File

In [4]:
## Function call to extract RVI Values 

time_slice = "2021-12-01/2022-08-30"
assests = ['vh','vv']
rvi = []
for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    #print("return",get_sentinel_data(coordinates,time_slice,assests))
    rvi.append(get_sentinel_data(coordinates,time_slice,assests))


100%|██████████| 600/600 [1:44:00<00:00, 10.40s/it]


In [5]:
# Covert list containing RVI values into Dataframe
rvi_data = pd.DataFrame(rvi,columns = ['rvi1','rvi2','rvi3','rvi4','rvi5',
                                          'rvi6','rvi7','rvi8','rvi9','rvi10',
                                          'rvi11','rvi12','rvi13','rvi14','rvi15',
                                          'rvi16','rvi17','rvi18', 'rvi19','rvi20',
                                          'rvi21','rvi22','rvi23','rvi24','rvi25',
                                          'rvi26','rvi27','rvi28','rvi29','rvi30',
                                          'rvi31','rvi32','rvi33','rvi34','rvi35',
                                          'rvi36','rvi37','rvi38','rvi39','rvi40',
                                          'rvi41','rvi42','rvi43','rvi44','rvi45',
                                          'rvi46'] )

In [6]:
#print first 5 rows of data
rvi_data.head()

,rvi1,rvi2,rvi3,rvi4,rvi5,rvi6,rvi7,rvi8,rvi9,rvi10,...,rvi37,rvi38,rvi39,rvi40,rvi41,rvi42,rvi43,rvi44,rvi45,rvi46
0,1.111368,0.617824,0.698573,0.394425,0.674143,0.316586,0.466150,0.582673,0.184040,0.279934,...,0.993113,0.602891,0.552731,0.603145,0.655351,0.432936,0.471715,0.236284,0.290553,NaN
1,0.654609,0.787948,0.687906,0.354302,0.465752,0.769894,0.456114,0.268524,0.598958,0.490678,...,0.921718,0.932209,0.592974,0.850943,0.484364,0.528684,0.908461,0.188856,0.087447,NaN
2,0.890117,0.867719,0.379095,0.443689,0.340729,0.343260,0.301123,0.571421,0.209795,0.383820,...,1.010928,1.036065,0.618302,1.024155,0.694684,0.766198,0.700091,0.145994,0.151130,NaN
3,0.638106,0.700709,0.435121,0.505765,0.532921,0.408293,0.462448,0.517581,0.177932,0.107987,...,0.868383,0.607660,0.491217,0.574914,0.268481,0.512498,0.629856,0.383905,0.137984,NaN
4,0.775777,0.675220,0.698803,0.343135,0.426485,0.436262,0.523983,0.487967,0.091241,0.136492,...,0.673819,0.685046,0.668283,0.895021,0.640925,0.602882,0.699218,0.147155,0.162441,NaN


In [7]:
# Column "rvi46" has NaN values so dropping that column
rvi_data = rvi_data.drop("rvi46",axis=1)

In [8]:
rvi_data.head()

,rvi1,rvi2,rvi3,rvi4,rvi5,rvi6,rvi7,rvi8,rvi9,rvi10,...,rvi36,rvi37,rvi38,rvi39,rvi40,rvi41,rvi42,rvi43,rvi44,rvi45
0,1.111368,0.617824,0.698573,0.394425,0.674143,0.316586,0.466150,0.582673,0.184040,0.279934,...,0.589837,0.993113,0.602891,0.552731,0.603145,0.655351,0.432936,0.471715,0.236284,0.290553
1,0.654609,0.787948,0.687906,0.354302,0.465752,0.769894,0.456114,0.268524,0.598958,0.490678,...,0.951499,0.921718,0.932209,0.592974,0.850943,0.484364,0.528684,0.908461,0.188856,0.087447
2,0.890117,0.867719,0.379095,0.443689,0.340729,0.343260,0.301123,0.571421,0.209795,0.383820,...,1.058681,1.010928,1.036065,0.618302,1.024155,0.694684,0.766198,0.700091,0.145994,0.151130
3,0.638106,0.700709,0.435121,0.505765,0.532921,0.408293,0.462448,0.517581,0.177932,0.107987,...,0.873561,0.868383,0.607660,0.491217,0.574914,0.268481,0.512498,0.629856,0.383905,0.137984
4,0.775777,0.675220,0.698803,0.343135,0.426485,0.436262,0.523983,0.487967,0.091241,0.136492,...,0.908775,0.673819,0.685046,0.668283,0.895021,0.640925,0.602882,0.699218,0.147155,0.162441


In [9]:
def combine_two_datasets(dataset1,dataset2):
    '''
    Returns a  vertically concatenated dataset.
    Attributes:
    dataset1 - Dataset 1 to be combined 
    dataset2 - Dataset 2 to be combined
    '''
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [10]:
crop_data = combine_two_datasets(crop_presence_data,rvi_data)
crop_data.head()

,Latitude and Longitude,Class of Land,rvi1,rvi2,rvi3,rvi4,rvi5,rvi6,rvi7,rvi8,...,rvi36,rvi37,rvi38,rvi39,rvi40,rvi41,rvi42,rvi43,rvi44,rvi45
0,"(10.323727047081501, 105.2516346045924)",Rice,1.111368,0.617824,0.698573,0.394425,0.674143,0.316586,0.466150,0.582673,...,0.589837,0.993113,0.602891,0.552731,0.603145,0.655351,0.432936,0.471715,0.236284,0.290553
1,"(10.322364360592521, 105.27843410554115)",Rice,0.654609,0.787948,0.687906,0.354302,0.465752,0.769894,0.456114,0.268524,...,0.951499,0.921718,0.932209,0.592974,0.850943,0.484364,0.528684,0.908461,0.188856,0.087447
2,"(10.321455902933202, 105.25254306225168)",Rice,0.890117,0.867719,0.379095,0.443689,0.340729,0.343260,0.301123,0.571421,...,1.058681,1.010928,1.036065,0.618302,1.024155,0.694684,0.766198,0.700091,0.145994,0.151130
3,"(10.324181275911162, 105.25118037576274)",Rice,0.638106,0.700709,0.435121,0.505765,0.532921,0.408293,0.462448,0.517581,...,0.873561,0.868383,0.607660,0.491217,0.574914,0.268481,0.512498,0.629856,0.383905,0.137984
4,"(10.324635504740822, 105.27389181724476)",Rice,0.775777,0.675220,0.698803,0.343135,0.426485,0.436262,0.523983,0.487967,...,0.908775,0.673819,0.685046,0.668283,0.895021,0.640925,0.602882,0.699218,0.147155,0.162441


In [11]:
# Save file as csv for future use
crop_data.to_csv('crop_data_senti1_rvi_45Columns_2021-12-01_2022-08-30.csv',index = False)  

## RVI Data Gathering for "challenge_1_submission_template_correct_columns_fixed" File

In [12]:
#Reading the coordinates for the submission file
test_file = pd.read_csv('challenge_1_submission_template_correct_columns_fixed.csv') 
print(test_file.shape)
test_file.head() 

(250, 2)


,id,target
0,"(10.18019073690894, 105.32022315786804)",NaN
1,"(10.561107033461816, 105.12772097986661)",NaN
2,"(10.623790611954897, 105.13771401411867)",NaN
3,"(10.583364246115156, 105.23946127195805)",NaN
4,"(10.20744446668854, 105.26844107128906)",NaN


In [13]:
## Function call to extract RVI Values
time_slice = "2021-12-01/2022-08-30" 
assests = ['vh','vv']
rvi = []
for coordinates in tqdm(test_file['id']): #Latitude and Longitude
    rvi.append(get_sentinel_data(coordinates,time_slice,assests))


100%|██████████| 250/250 [43:22<00:00, 10.41s/it]


In [14]:
submission_rvi_data = pd.DataFrame(rvi,columns = ['rvi1','rvi2','rvi3','rvi4','rvi5',
                                          'rvi6','rvi7','rvi8','rvi9','rvi10',
                                          'rvi11','rvi12','rvi13','rvi14','rvi15',
                                          'rvi16','rvi17','rvi18', 'rvi19','rvi20',
                                          'rvi21','rvi22','rvi23','rvi24','rvi25',
                                          'rvi26','rvi27','rvi28','rvi29','rvi30',
                                          'rvi31','rvi32','rvi33','rvi34','rvi35',
                                          'rvi36','rvi37','rvi38','rvi39','rvi40',
                                          'rvi41','rvi42','rvi43','rvi44','rvi45'] )

In [15]:
submission_rvi_data.head()

,rvi1,rvi2,rvi3,rvi4,rvi5,rvi6,rvi7,rvi8,rvi9,rvi10,...,rvi36,rvi37,rvi38,rvi39,rvi40,rvi41,rvi42,rvi43,rvi44,rvi45
0,1.265666,1.040062,0.953330,0.481993,0.419678,0.333645,0.319963,0.260668,0.197258,0.239706,...,0.785152,0.734679,0.858956,0.932741,0.718733,0.388959,0.352628,0.342057,0.365078,0.493889
1,0.884043,0.690166,1.170694,0.689360,0.637310,0.308467,0.289741,1.135938,0.410704,0.626633,...,0.866087,0.723486,0.857754,0.888209,0.928014,0.479017,0.459849,0.537319,0.709433,0.724080
2,0.451890,0.459892,0.542179,0.284459,0.342210,0.546757,0.196962,0.168028,0.212754,0.236922,...,0.754909,0.965200,0.867237,0.533282,0.676945,0.820860,0.804863,0.380770,0.710452,0.808880
3,0.600245,0.841846,0.862757,0.778806,0.827842,0.744846,0.939001,0.984508,0.777914,0.918527,...,0.856625,0.747937,0.868692,0.688991,1.114499,0.725051,0.726753,0.569186,0.480176,1.128619
4,0.334199,0.483108,0.270834,0.142687,0.196492,0.162851,0.366315,0.379547,0.813936,1.210913,...,0.989964,1.018154,0.491679,0.337446,0.550406,0.467019,0.596292,0.345243,0.651412,0.531299


In [16]:
# Save dataframe to CSV file for future use
submission_rvi_data.to_csv('submission_vh_vv_data_rvi_45Columns_2021-12-01_2022-08-30.csv') 